In [1]:
import pandas as pd

In [17]:
df = pd.read_csv('jeopardy-topic-modeling/JEOPARDY_CSV.csv')
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [18]:
df[' Round'].unique()

array(['Jeopardy!', 'Double Jeopardy!', 'Final Jeopardy!', 'Tiebreaker'],
      dtype=object)

In [19]:
len(df[df[' Round'] == 'Final Jeopardy!'])

3631

### Clean up data

In [20]:
df.columns = ['show_id', 'date', 'round', 'category', 'value', 'question', 'answer']

In [21]:
#df.value.unique()

In [22]:
#df[df.value == 'None']

In [23]:
df['value'].replace({'None': '$0'}, inplace=True)

In [24]:
df['value'] = df['value'].apply(lambda x: int(x[1:].replace(',',''))) #remove $ remove comma and convert to int

In [26]:
df['year'] = df['date'].apply(lambda x: int(x[:4]))

In [27]:
df.head()

,show_id,date,round,category,value,question,answer,year
0,4680,2004-12-31,Jeopardy!,HISTORY,200,"For the last 8 years of his life, Galileo was ...",Copernicus,2004
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,2004
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200,The city of Yuma in this state has a record av...,Arizona,2004
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,2004
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,2004


In [29]:
df['round'].unique()

array(['Jeopardy!', 'Double Jeopardy!', 'Final Jeopardy!', 'Tiebreaker'],
      dtype=object)

### Explore categories

In [41]:
df.category.value_counts()[:25]

BEFORE & AFTER             547
SCIENCE                    519
LITERATURE                 496
AMERICAN HISTORY           418
POTPOURRI                  401
WORLD HISTORY              377
WORD ORIGINS               371
COLLEGES & UNIVERSITIES    351
HISTORY                    349
SPORTS                     342
U.S. CITIES                339
WORLD GEOGRAPHY            338
BODIES OF WATER            327
ANIMALS                    324
STATE CAPITALS             314
BUSINESS & INDUSTRY        311
ISLANDS                    301
WORLD CAPITALS             300
U.S. GEOGRAPHY             299
RELIGION                   297
OPERA                      294
SHAKESPEARE                294
LANGUAGES                  284
BALLET                     282
TELEVISION                 281
Name: category, dtype: int64

## Attempt at running LDA

In [50]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joshuamajano/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [78]:
test = df[df.year==1984]
test.shape

(1179, 8)

In [79]:
data_text = test[['question']]

In [80]:
data_text.head()

,question
8544,"What the ""D.C."" stands for"
8545,"Color of a lucky ""letter day"""
8546,"While 2 of its wheels head toward produce, 1 g..."
8547,Varieties include Chinese & Dijon
8548,"Turned down to be an ""Our Gang"" member, she be..."


In [82]:
#data_text

In [84]:
data_text.iloc[0]['question'].lower().split()

['what', 'the', '"d.c."', 'stands', 'for']

In [86]:
for idx in range(len(data_text)):
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['question'] = [word for word in data_text.iloc[idx]['question'].lower().split(' ') if word not in stopwords.words('english')]

In [87]:
data_text.head()

,question
8544,"[""d.c."", stands]"
8545,"[color, lucky, ""letter, day""]"
8546,"[2, wheels, head, toward, produce,, 1, goes, d..."
8547,"[varieties, include, chinese, &, dijon]"
8548,"[turned, ""our, gang"", member,, became, top, bo..."


In [88]:
train_headlines = [value[0] for value in data_text.iloc[0:].values]

In [95]:
#train_headlines

In [90]:
id2word = gensim.corpora.Dictionary(train_headlines)

In [91]:
corpus = [id2word.doc2bow(text) for text in train_headlines]

In [92]:
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10)

In [93]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

get_lda_topics(lda, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,&,1st,&,&,state,name,&,&,&,&
1,1st,1,name,name,&,john,first,years,country,made
2,u.s.,&,call,"""the",1st,used,played,film,number,song
3,"""the",2,country,came,became,people,named,name,largest,<a
4,name,called,comes,president,famous,one,term,france,world's,time
5,4,body,named,greek,first,called,3,include,name,new
6,state,"""the",roman,call,british,last,day,called,though,type
7,first,that's,people,black,home,still,number,1st,america,first
8,though,american,whose,best,country,crew,though,old,could,largest
9,became,bird,river,africa's,city,british,president,8,war,french


In [94]:
# Taken from article

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
#import sys
from nltk.corpus import stopwords;
import nltk
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
#import pickle;


# # Loading the data
# 
# We are using the ABC News headlines dataset. Some lines are badly formatted (very few), so we are skipping those.

#We only need the Headlines_text column from the data
data_text = data[['headline_text']];


# We need to remove stopwords first. Casting all values to float will make it easier to iterate over.

data_text = data_text.astype('str');

for idx in range(len(data_text)):
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['headline_text'] = [word for word in data_text.iloc[idx]['headline_text'].split(' ') if word not in stopwords.words()];
    
    #print logs to monitor output
    #if idx % 1000 == 0:
    #    sys.stdout.write('\rc = ' + str(idx) + ' / ' + str(len(data_text)));

#save data because it takes very long to remove stop words
#pickle.dump(data_text, open('data_text.dat', 'wb'))

#get the words as an array for lda input
train_headlines = [value[0] for value in data_text.iloc[0:].values];

#number of topics we will cluster for: 10
num_topics = 10;


# # LDA
# We will use the gensim library for LDA. First, we obtain a id-2-word dictionary. 
#For each headline, we will use the dictionary to obtain a mapping of the word id to their word counts. 
#The LDA model uses both of these mappings.

id2word = gensim.corpora.Dictionary(train_headlines);

corpus = [id2word.doc2bow(text) for text in train_headlines];

lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics);


# # generating LDA topics
# 
# We will iterate over the number of topics, get the top words in each cluster and add them to a dataframe.

def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

get_lda_topics(lda, num_topics)



# # NMF
# For NMF, we need to obtain a design matrix. To improve results, I am going to apply TfIdf 
#transformation to the counts.

#the count vectorizer needs string inputs, not array, so I join them with a space.
train_headlines_sentences = [' '.join(text) for text in train_headlines]


# Now, we obtain a Counts design matrix, for which we use SKLearn’s CountVectorizer module. 
#The transformation will return a matrix of size (Documents x Features), where the value of a cell 
#is going to be the number of times the feature (word) appears in that document.
# To reduce the size of the matrix, to speed up computation, we will set the maximum feature size 
#to 5000, which will take the top 5000 best features that can contribute to our model.

vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(train_headlines_sentences);


# Next, we set a TfIdf Transformer, and transform the counts with the model.

transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);


# And now we normalize the TfIdf values to unit length for each row.

xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)


# And finally, obtain a NMF model, and fit it with the sentences.

#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');

#fit the model
model.fit(xtfidf_norm)

def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

get_nmf_topics(model, 20)